In [19]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itr
import time
from boundaries import WallBoundary

# Initializing a rank three tensor filled with nodes
latticeSize = 9
xResolution = 5
yResolution = 5
relaxationTime = 0.5
# Weights
unitVect = np.array([
	[0, 0],	[1, 0], [0, 1],
	[-1, 0], [0, -1], [1, 1],
	[-1, 1], [-1, -1], [1, -1]
])
unitX = np.array([
	0, 1, 0, -1, 0, 1, -1, -1, 1
])
unitY = np.array([
	0, 0, 1, 0, -1, 1, 1, -1, -1
])
weight = np.array([
    4/9, 1/9, 1/9,
    1/9, 1/9, 1/36,
    1/36, 1/36, 1/36
])
cs = np.sqrt(3)

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 3)

In [20]:
# One iteration only
iteratedFluid = np.ones((xResolution, yResolution, latticeSize))

# Internal collision step
density = np.sum(fluid, axis = 2)
mesoMomentum = fluid.reshape(xResolution, yResolution, latticeSize, 1) * unitVect
momentum = np.sum(mesoMomentum, axis = 2) / density.reshape(xResolution, yResolution, 1)
momentumMagnitude = np.linalg.norm(momentum, axis = 2) ** 2 / (2 * cs**4) # Represents u . u

# I kinda give up on broadcasting rn. Help me :skull:
for xIndex, yIndex in itr.product(range(xResolution), range(yResolution)):
    latticeDensity = density[xIndex, yIndex]
    cellMomentumMagnitude = momentumMagnitude[xIndex, yIndex]
    for latticeIndex in range(latticeSize):
        momentumDot = np.dot(unitVect[latticeIndex], momentum[xIndex, yIndex])
        fluidEquilibrium = latticeDensity * weight[latticeIndex] * (
			1 + momentumDot / (cs ** 2) + momentumDot ** 2 / (cs ** 4) - cellMomentumMagnitude
		)
        iteratedFluid[xIndex, yIndex, latticeIndex] = fluid[xIndex, yIndex, latticeIndex] * (1 - 1/relaxationTime) + 1/relaxationTime * fluidEquilibrium

iteratedFluid

array([[[ 8.47572874e+00,  1.51057429e-01,  1.39166008e+00,
          1.05862676e+00,  1.21963956e+00, -2.95647826e-01,
         -5.96019245e-01, -3.08060285e-01, -3.52568549e-01],
        [ 7.78835153e+00,  2.34444181e-02,  1.28274971e+00,
          1.33071769e+00,  1.21312079e+00, -4.87366539e-01,
         -4.78536342e-01, -4.36847517e-01, -3.12890632e-01],
        [ 7.74527697e+00,  1.47459109e-02,  1.40006245e+00,
          1.12154677e+00,  1.37272985e+00, -4.29753826e-01,
         -4.16779673e-01, -4.58405497e-01, -4.09777712e-01],
        [ 8.72228718e+00,  1.95060960e-01,  1.46334663e+00,
          1.19704399e+00,  1.34875393e+00, -3.06287052e-01,
         -6.05892003e-01, -5.97269725e-01, -4.16448077e-01],
        [ 8.66528264e+00,  1.89474067e-01,  1.35298731e+00,
          1.28503290e+00,  1.38264867e+00, -5.32192487e-01,
         -4.53880710e-01, -6.61934812e-01, -4.50278154e-01]],

       [[ 8.32550733e+00,  1.43192037e-01,  1.39133393e+00,
          9.92947973e-01,  1.3448

In [21]:
boundary.boundary

array([[False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [22]:
for latticeIndex, shiftX, shiftY in zip(range(latticeSize), unitX, unitY):
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftX, axis = 1)
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftY, axis = 0)

fluidBoundary = boundary.boundary

for x, biglist in enumerate(fluidBoundary):
    for y, isAWall in enumerate(biglist):
        if (not isAWall):
            for latticeIndex in range(latticeSize):
                #for each vector in the lattice, send it back
                if (iteratedFluid[x, y, latticeIndex] != 0):
                    if latticeIndex in [3, 4, 7, 8]:
                        reflectIndex = latticeIndex - 2
                    else: 
                        reflectIndex = latticeIndex + 2
                    
                    iteratedFluid[x -  unitX[latticeIndex], y - unitY[latticeIndex], reflectIndex] = iteratedFluid[x, y, latticeIndex]

iteratedFluid

array([[[ 8.47572874e+00,  1.89474067e-01,  8.47572874e+00,
          1.33071769e+00,  1.34486405e+00, -5.14196150e-01,
         -4.98098836e-01, -5.82689284e-01, -2.61811578e-01],
        [ 7.78835153e+00,  1.51057429e-01,  1.34486405e+00,
          1.12154677e+00,  1.26784255e+00, -2.95945710e-01,
         -5.34924448e-01, -4.54339675e-01, -5.34714940e-01],
        [ 7.74527697e+00,  2.34444181e-02,  1.25234746e+00,
          1.19704399e+00,  1.26327380e+00, -4.82530211e-01,
         -4.15789767e-01, -3.47992778e-01, -3.77291121e-01],
        [ 8.72228718e+00,  1.47459109e-02,  1.23067874e+00,
          1.28503290e+00,  1.21754186e+00, -4.59560165e-01,
         -4.95112384e-01, -3.58736289e-01, -2.82416356e-01],
        [ 8.66528264e+00,  1.95060960e-01,  1.31957347e+00,
          1.05862676e+00,  8.47572874e+00, -3.62157661e-01,
         -5.92061303e-01, -3.89196135e-01, -4.18285360e-01]],

       [[ 8.32550733e+00,  1.33071769e+00,  1.39166008e+00,
          1.21373011e+00,  1.3477